In [18]:
import pandas as pd
import math
import os
import random
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [19]:
class MovieDataLoader:
    def __init__(self, file_path = '../data'):
        self.file_path = file_path
        self.__load_data()

    def __load_data(self):
        for file_name in os.listdir(self.file_path):
            self.file_name = []
            if file_name.endswith('.csv'):
                setattr(self, os.path.splitext(file_name)[0], pd.read_csv(os.path.join(self.file_path, file_name)))
            if file_name.endswith('.txt'):
                setattr(self, os.path.splitext(file_name)[0], pd.read_csv(os.path.join(self.file_path, file_name), sep='\t', engine='python'))


class RandomRecommender:
    def __init__(self, movie_data_loader):
        self.loader = movie_data_loader
        self.movies = self.loader.movies
        self.rates = self.loader.rates

    def run(self):
        self.rates['rate_random'] = [random.uniform(1, 10) for _ in range(len(self.rates))]
        self.rates = self.rates.sort_values(by='rate_random', ascending=False)
        # 사사오입 반올림
        self.rates['rate_random_class'] = self.rates['rate_random'].apply(lambda x: math.floor(x + 0.5))
        return self
    
class Analyzer:
    def __init__(self, recommender):
        self.recommender = recommender
        self.rates = self.recommender.rates

    def evaluate(self):
        self.MAE = sum(abs(self.rates['rate'] - self.rates['rate_random'])) / len(self.rates)
        self.MSE = sum((self.rates['rate'] - self.rates['rate_random']) ** 2) / len(self.rates)
        self.RMSE = self.MSE ** 0.5
        self.MAPE = sum(abs(self.rates['rate'] - self.rates['rate_random']) / self.rates['rate']) / len(self.rates)
        self.ConfusionMatrix = self.rates.groupby(['rate', 'rate_random_class']).size().unstack(fill_value=0)
        self.Accuracy = sum([self.ConfusionMatrix.loc[i, i] for i in range(1, 11)]) / len(self.rates)
        self.Precision = [self.ConfusionMatrix.loc[i, i] / sum(self.ConfusionMatrix.loc[i]) for i in range(1, 11)]
        self.Recall = [self.ConfusionMatrix.loc[i, i] / sum(self.ConfusionMatrix.loc[i]) for i in range(1, 11)]
        self.F1Score = [2 * self.Precision[i] * self.Recall[i] / (self.Precision[i] + self.Recall[i]) for i in range(0, 10)]
        print(f"ConfusionMatrix: {self.ConfusionMatrix}")
        print(f"Accuracy: {self.Accuracy}")
        print(f"Precision: {self.Precision}")
        print(f"Recall: {self.Recall}")
        print(f"F1Score: {self.F1Score}")
        return self
    
movie_data_loader = MovieDataLoader("../data")
random_recommender = RandomRecommender(movie_data_loader)
random_recommender.run()

analyzer = Analyzer(random_recommender)
analyzer.evaluate()


ConfusionMatrix: rate_random_class    1      2      3      4      5      6      7      8   \
rate                                                                       
1                   264    592    582    602    573    543    601    586   
2                    38     75    106     82     84     70     93     79   
3                    48    102    107     78     96     84     92     93   
4                    55    110    142    120    108    126    118    120   
5                   100    218    249    251    224    223    216    239   
6                   209    411    404    393    416    380    431    406   
7                   364    710    743    729    664    724    749    760   
8                   595   1274   1285   1269   1326   1269   1339   1261   
9                   817   1703   1642   1597   1663   1580   1620   1612   
10                 5328  10535  10559  10436  10503  10481  10374  10519   

rate_random_class     9     10  
rate                            
1   

# 혼동 행렬 (Confusion Matrix)

# 정확도 (Accuracy)

# 정밀도 (Precision)

# 재현율 (Recall)

# F1 점수 (F1 Score)

# + ROC 곡선 (ROC Curve)

# REF
[Simplifying The Visualization of Confusion Matix](https://ir.cwi.nl/pub/22777/22777B.pdf)

[조화평균 이해](https://www.youtube.com/watch?v=llK4YBiv4uc)

# INFO
python 반올림 방법
Bankers' Rounding 형식, 우리는 사사오입방식으로 해야. class의 분산의 영향력을 최소화 할 수 있다.